In [1]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier


### MNIST con GradientBoosting
En esta segunda sesión práctica vamos a entrenar clasificadores basados en GradientBoosting.
Claramente estos clasificadores son más costosos de entrenar.


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)

c:\Users\adri_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [3]:
# Partición de los datos
X_train, X_aux, y_train, y_aux = train_test_split(data, targets, test_size=0.2, shuffle=True, random_state=23)
X_test, X_val, y_test, y_val = train_test_split(X_aux, y_aux, test_size=0.5, shuffle=True, random_state=23)

X_tv = np.concatenate((X_train, X_val))
y_tv = np.concatenate((y_train, y_val))

### **Ejercicio 2**   

Prueba un clasificador GradientBoosting

In [4]:
## GradientBoosting
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
y_pred = gb.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f'La precisión de {gb} es {acc:.1%}')
# Ha tardado 78m 31.5s

La precisión de GradientBoostingClassifier() es 95.1%


Podrás comprobar como es un clasificador mucho más lento. 

### **Ejercicio 3**   

Para reducir el coste computacional se propone crear un pipeline donde se reduzca el número de características mediante PCA. En concreto el número de componentes (dimensiones) a las que reducimos con PCA es un hyperparámetro que tendrás que estimar con el conjunto de validación.

In [4]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

In [8]:
## Mejorar velocidad reduciendo dimensionalidad con PCA
max = -1
best_comp = 0
for nc in [2,4,8,16,32]:
    clf = make_pipeline(
        PCA(n_components=nc),
        GradientBoostingClassifier()
    )

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)

    if acc > max:
        max = acc
        best_comp = nc
        print(f'La nueva precisión máxima de {clf} es {acc:.1%}')


La nueva precisión máxima de Pipeline(steps=[('pca', PCA(n_components=2)),
                ('gradientboostingclassifier', GradientBoostingClassifier())]) es 47.4%
La nueva precisión máxima de Pipeline(steps=[('pca', PCA(n_components=4)),
                ('gradientboostingclassifier', GradientBoostingClassifier())]) es 64.7%
La nueva precisión máxima de Pipeline(steps=[('pca', PCA(n_components=8)),
                ('gradientboostingclassifier', GradientBoostingClassifier())]) es 86.2%
La nueva precisión máxima de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('gradientboostingclassifier', GradientBoostingClassifier())]) es 91.0%
La nueva precisión máxima de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('gradientboostingclassifier', GradientBoostingClassifier())]) es 92.3%


De entre los diferentes parámeros que tiene el clasificador GradientBoosting de sklearn, cabría destacar:

**learning_rate**

**n_estimators**

**min_samples_split**

**max_features** 


Para más información leer la documentación en sklearn.

Alguno de estos parámetros influyen considerablemente en la velocidad de optimización. Por ejemplo **max_features** y **min_samples_split** entre otros.



### **Ejercicio 4**   

Se propone variar alguno de estos parámetros para ver si se obtiene una similar tasa de acierto pero con mejor velocidad.


In [6]:
# Calcular tiempo y tasa de acierto de la versión original con PCA 
import time

clf = make_pipeline(
    PCA(n_components=best_comp),
    GradientBoostingClassifier()
)

t0 = time.time()
clf.fit(X_tv, y_tv)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
t1 = time.time()
print(f'La precisión de {clf} es {acc:.1%} y tarda {(t1 - t0):.2} segundos')

La precisión de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('gradientboostingclassifier', GradientBoostingClassifier())]) es 92.5% y tarda 1.4e+03 segundos


In [23]:
# Calcular tiempo y tasa de acierto modificando alguno de los parámetros propuestos
import math

max = -1
best_comp=16
best_mss = 10
max_features = round(math.sqrt(784))
# for mss in [2,4,8,16,32]:

clf = make_pipeline(
    PCA(n_components=best_comp),
    GradientBoostingClassifier(min_samples_split=best_mss, max_features=max_features)
)

t0 = time.time()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val, y_pred)
t1 = time.time()

# if acc > max:
#     max = acc
#     best_mss = mss

print(f'La precisión de {clf} es {acc:.1%} y tarda {(t1 - t0):.2} segundos')

La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(max_features=28,
                                            min_samples_split=10))]) es 90.9% y tarda 6.4e+02 segundos


Finalmente sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma: el HistGradientBoostingClassifier. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



### **Ejercicio 5**   

Pruébalo y compara los tiempos de ejecución.

In [27]:
# Probar HistGradientBoostingClassifier 
from sklearn.ensemble import HistGradientBoostingClassifier

clf = make_pipeline(
    PCA(n_components=32),
    HistGradientBoostingClassifier()
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f'La precisión de {clf} es {acc:.1%}')

La precisión de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())]) es 96.6%


### **Ejercicio 6**   

Prueba los parámetros del HistGradientBoostingClassifier que mejoren la tasa de acierto. En cualquier caso la selección de estos parámetros debe seguir el protocolo de experimentación antes expuesto. Esto es, escoger el mejor parámetro con datos de validación y reportar resultados con los datos de test.

In [34]:
max_a = -1
best_nc = 0

for nc in [2,4,8,16,32,64]:
    clf = make_pipeline(
        PCA(n_components=nc),
        HistGradientBoostingClassifier()
    )

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)

    if acc > max_a:
        max_a = acc
        best_nc = nc

    print(f'La precisión de {clf} es {acc:.1%}')

La precisión de Pipeline(steps=[('pca', PCA(n_components=2)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())]) es 47.0%
La precisión de Pipeline(steps=[('pca', PCA(n_components=4)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())]) es 66.2%
La precisión de Pipeline(steps=[('pca', PCA(n_components=8)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())]) es 90.1%
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())]) es 95.1%
La precisión de Pipeline(steps=[('pca', PCA(n_components=32)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())]) es 96.5%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
         

In [6]:
from sklearn.ensemble import HistGradientBoostingClassifier

max = -1
for nc in [16,32,64]:
    for max_depth in [8,16,32,64,128]:
        for msl in [16,32,64,128,256]:
            for mb in [64,80,90,100,110,128]:
                for lr in [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]:
                    for l2 in [0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1]:
                        clf = make_pipeline(
                            PCA(n_components=nc),
                            HistGradientBoostingClassifier(max_depth=max_depth, min_samples_leaf=msl, max_bins=mb, learning_rate=lr, l2_regularization=l2)
                        )
                        clf.fit(X_train, y_train)
                        y_pred = clf.predict(X_val)
                        acc = accuracy_score(y_val, y_pred)

                        print(f'---PARAMS: n_components={nc} max_depth={max_depth}, min_samples_leaf={msl}, max_bins={mb}, learning_rate={lr}, l2_regularization={l2}, ACC={acc:.2%}')
                        if acc > max:
                            max = acc
                            print(f'\n---La NUEVA PRECISIÓN MÁXIMA de {clf} es {acc:.2%}\n')

---PARAMS: n_components=16 max_depth=8, min_samples_leaf=16, max_bins=64, learning_rate=0.1, l2_regularization=0, ACC=95.01%

---La NUEVA PRECISIÓN MÁXIMA de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(l2_regularization=0,
                                                max_bins=64, max_depth=8,
                                                min_samples_leaf=16))]) es 95.01%

---PARAMS: n_components=16 max_depth=8, min_samples_leaf=16, max_bins=64, learning_rate=0.1, l2_regularization=0.1, ACC=95.16%

---La NUEVA PRECISIÓN MÁXIMA de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(l2_regularization=0.1,
                                                max_bins=64, max_depth=8,
                                                min_samples_leaf=16))]) es 95.16%

---PARAMS: n_components=16 max_d

KeyboardInterrupt: 

In [37]:
for mb in [64,80,90,100,110,128]:
    clf = make_pipeline(
        PCA(n_components=best_nc),
        HistGradientBoostingClassifier(max_bins=mb, max_depth=64)
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f'La precisión de {clf} es {acc:.1%}')

La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=64, max_depth=64))]) es 96.7%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=80, max_depth=64))]) es 96.5%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=90, max_depth=64))]) es 96.6%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=100, max_depth=64))]) es 96.6%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=110, max_depth=64))]) es

In [39]:
for mss in [16,32,64,128,256]:
    clf = make_pipeline(
        PCA(n_components=best_nc),
        HistGradientBoostingClassifier(max_bins=64, max_depth=64, min_samples_leaf=mss)
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f'La precisión de {clf} es {acc:.1%}')

La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=64, max_depth=64,
                                                min_samples_leaf=16))]) es 96.3%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=64, max_depth=64,
                                                min_samples_leaf=32))]) es 96.7%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=64, max_depth=64,
                                                min_samples_leaf=64))]) es 96.6%
La precisión de Pipeline(steps=[('pca', PCA(n_components=64)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=64, max_depth=64,
 